**Acknowledgement**

The codes, data, tutorial flow are from [Introduction to Mathematical Optimization Modeling]("https://www.gurobi.com/jupyter_models/introduction-to-mathematical-optimization-modeling/") from Gurobi.

I modified some codes and added MCNFP part.
For example, codes to check model specification and boundaries for decision variables are parts where I modified.

**Here I fiddeled with Gurobi to see how to formulate an assignment problem as a minimum cost network flow problem (MCNFP).**

Three open positions at a company

1. Tester
2. Java Developer
3. Architect

Three candidates (= resources)

1. Carlos
2. Joe
3. Monika

Assignment probelm based on test scores. Three candidates take tests for each role, so there are in total 9 test scores that can fit into 3 * 3 table.

(Note that this is an assignment problem which can be formulated as MCNFP.)

In [1]:
#!pip install gurobipy

In [2]:
import gurobipy as gp
from gurobipy import GRB

In [3]:
#Assignment: Resources to Jobs

#resources
R = ['Carlos', 'Joe', 'Monika']

#jobs
J = ['Tester', 'JavaDeveloper', 'Architect']

In [4]:
#Test result (python dictionary)
#This data is given

test_scores = {('Carlos', 'Tester'): 53, 
               ('Carlos', 'JavaDeveloper'): 27, 
               ('Carlos', 'Architect'): 13,
               ('Joe', 'Tester'): 80,
               ('Joe', 'JavaDeveloper'): 47,
               ('Joe', 'Architect'): 67,
               ('Monika', 'Tester'): 53,
               ('Monika', 'JavaDeveloper'): 73,
               ('Monika', 'Architect'): 47}

In [5]:
#gurobi multidict function
#combination = arc = decision variable
#score = cost for each arc

combinations, scores = gp.multidict(test_scores)

In [6]:
print(combinations)
print(scores)

<gurobi.tuplelist (9 tuples, 2 values each):
 ( Carlos , Tester        )
 ( Carlos , JavaDeveloper )
 ( Carlos , Architect     )
 ( Joe    , Tester        )
 ( Joe    , JavaDeveloper )
 ( Joe    , Architect     )
 ( Monika , Tester        )
 ( Monika , JavaDeveloper )
 ( Monika , Architect     )
>
{('Carlos', 'Tester'): 53, ('Carlos', 'JavaDeveloper'): 27, ('Carlos', 'Architect'): 13, ('Joe', 'Tester'): 80, ('Joe', 'JavaDeveloper'): 47, ('Joe', 'Architect'): 67, ('Monika', 'Tester'): 53, ('Monika', 'JavaDeveloper'): 73, ('Monika', 'Architect'): 47}


In [7]:
# Model Initialization
#'RAP' is the name of the model in short for 'Resource Assignement Problem'

m = gp.Model('RAP')

Set parameter Username


In [8]:
# Create decision variables
x = m.addVars(combinations, vtype=GRB.BINARY, name = 'assign')

In [9]:
# Create Jobs (receiving nodes) constraints
# Each job should be assigned a person
jobs = m.addConstrs((x.sum('*',j) == 1 for j in J), name='job')

In [10]:
# Create Resources (assigning nodes) constraints
# Inequality -> suggests there can be a case where not all resources are assigned
resources = m.addConstrs((x.sum(r, '*') <= 1 for r in R), name='resource')

In [11]:
# Objective function (maximize the sum of test scores)
# Refer to print(scores) for better understanding
m.setObjective(x.prod(scores), GRB.MAXIMIZE)

In [12]:
# Make .lp file for running solver
m.write('RAP.lp')

In [13]:
with open('RAP.lp', 'r') as file:
    content = file.read()
    print(content)

\ Model RAP
\ LP format - for model browsing. Use MPS format to capture full model detail.
Maximize
  53 assign[Carlos,Tester] + 27 assign[Carlos,JavaDeveloper]
   + 13 assign[Carlos,Architect] + 80 assign[Joe,Tester]
   + 47 assign[Joe,JavaDeveloper] + 67 assign[Joe,Architect]
   + 53 assign[Monika,Tester] + 73 assign[Monika,JavaDeveloper]
   + 47 assign[Monika,Architect]
Subject To
 job[Tester]: assign[Carlos,Tester] + assign[Joe,Tester]
   + assign[Monika,Tester] = 1
 job[JavaDeveloper]: assign[Carlos,JavaDeveloper]
   + assign[Joe,JavaDeveloper] + assign[Monika,JavaDeveloper] = 1
 job[Architect]: assign[Carlos,Architect] + assign[Joe,Architect]
   + assign[Monika,Architect] = 1
 resource[Carlos]: assign[Carlos,Tester] + assign[Carlos,JavaDeveloper]
   + assign[Carlos,Architect] <= 1
 resource[Joe]: assign[Joe,Tester] + assign[Joe,JavaDeveloper]
   + assign[Joe,Architect] <= 1
 resource[Monika]: assign[Monika,Tester] + assign[Monika,JavaDeveloper]
   + assign[Monika,Architect] <= 1


In [14]:
# Run
m.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i7-10750H CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 6 rows, 9 columns and 18 nonzeros
Model fingerprint: 0x0a338f16
Variable types: 0 continuous, 9 integer (9 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 8e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.00s
Presolved: 6 rows, 9 columns, 18 nonzeros
Variable types: 0 continuous, 9 integer (9 binary)
Found heuristic solution: objective 193.0000000

Root relaxation: cutoff, 0 iterations, 0.00 seconds (0.00 work units)

Explored 1 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 12 (of 12 available processors)

Solution count 1: 193 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.930000000000

In [15]:
# Display optimal values of decision variables
for v in m.getVars():
    if v.x > 1e-6:
        print(v.varName, v.x)

# Display optimal total matching score
print('Total matching score: ', m.objVal)

assign[Carlos,Tester] 1.0
assign[Joe,Architect] 1.0
assign[Monika,JavaDeveloper] 1.0
Total matching score:  193.0


**Let's try alternative formulation as MCNFP**

In [16]:
m2 = gp.Model('MCNFP')

In [17]:
# Create decision variables (Note that binary condition is relaxed)
x2 = m2.addVars(combinations, lb=0, ub=1, name = 'assign')

In [18]:
# Create Jobs (receiving nodes) constraints
# Each job should be assigned a person
jobs2 = m2.addConstrs((-x2.sum('*',j) == -1 for j in J), name='job')

In [19]:
# Create Resources (assigning nodes) constraints
resources2 = m2.addConstrs((x2.sum(r, '*') == 1 for r in R), name='resource')

In [20]:
# Objective function (minimization of cost, cost = -score)
m2.setObjective(-x2.prod(scores), GRB.MINIMIZE)

In [21]:
# write .lp file
m2.write('MCNFP.lp')

In [22]:
#check model specification
with open('MCNFP.lp', 'r') as file:
    content = file.read()
    print(content)

\ Model MCNFP
\ LP format - for model browsing. Use MPS format to capture full model detail.
Minimize
  - 53 assign[Carlos,Tester] - 27 assign[Carlos,JavaDeveloper]
   - 13 assign[Carlos,Architect] - 80 assign[Joe,Tester]
   - 47 assign[Joe,JavaDeveloper] - 67 assign[Joe,Architect]
   - 53 assign[Monika,Tester] - 73 assign[Monika,JavaDeveloper]
   - 47 assign[Monika,Architect]
Subject To
 job[Tester]: - assign[Carlos,Tester] - assign[Joe,Tester]
   - assign[Monika,Tester] = -1
 job[JavaDeveloper]: - assign[Carlos,JavaDeveloper]
   - assign[Joe,JavaDeveloper] - assign[Monika,JavaDeveloper] = -1
 job[Architect]: - assign[Carlos,Architect] - assign[Joe,Architect]
   - assign[Monika,Architect] = -1
 resource[Carlos]: assign[Carlos,Tester] + assign[Carlos,JavaDeveloper]
   + assign[Carlos,Architect] = 1
 resource[Joe]: assign[Joe,Tester] + assign[Joe,JavaDeveloper]
   + assign[Joe,Architect] = 1
 resource[Monika]: assign[Monika,Tester] + assign[Monika,JavaDeveloper]
   + assign[Monika,Archi

In [23]:
m2.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i7-10750H CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 6 rows, 9 columns and 18 nonzeros
Model fingerprint: 0x59a0d829
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 8e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.00s
Presolved: 6 rows, 9 columns, 18 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -2.0600000e+02   2.000000e+00   0.000000e+00      0s
       1   -1.9300000e+02   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.01 seconds (0.00 work units)
Optimal objective -1.930000000e+02


In [24]:
# Display optimal values of decision variables
for v in m2.getVars():
    if v.x > 1e-6:
        print(v.varName, v.x)

# Display optimal total matching score
print('Total cost = - Total matching score: ', m2.objVal)

assign[Carlos,Tester] 1.0
assign[Joe,Architect] 1.0
assign[Monika,JavaDeveloper] 1.0
Total cost = - Total matching score:  -193.0
